In [1]:
import pandas as pd

from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

from pathfinding.finder.dijkstra import DijkstraFinder
from pathfinding.core.graph import Graph
from pathfinding.core.node import Node

import networkx as nx

In [3]:
def make_initial_result_dicts(points_grid_coords_dict, xls_schedule, xls_graph):
    

    applications = pd.read_excel(xls_schedule, sheet_name='Лист1', header=0)
    origin_time = applications['Дата начала плавания'].iloc[0]  #2022-03-01 00:00:00
    
    points = pd.read_excel(xls_graph, sheet_name='points', header=0)
    points['point_name'] = points['point_name'].apply(lambda x: str(x).lower())
    
    initial_ships_list = []
    
    for _, row in applications.iloc[:42].iterrows():
        
        start_place = str(row['Пункт начала плавания']).lower()
        
        try:
            if start_place == 'восточно-сибирское 1':
                start_place = 'Восточно-Сибирское - 1 (восток)'.lower()
            elif start_place == 'восточно-сибирское 3':
                start_place = 'Восточно-Сибирское - 3 (север)'.lower()
            start_place_id = points['point_id'][points['point_name'] == start_place].iloc[0]
        except:
            print(start_place)
            
        start_place_grid_coord = points_grid_coords_dict[start_place_id]
        ship = {
            "name_ship": row['Название судна'],
            "date_start": str(row['Дата начала плавания'].date()),
            "class_ship": row['Ледовый класс'],
            "speed_ship": row['''Скорость, узлы
(по чистой воде)'''],
            "route": [
                { 
                    'time': int((row['Дата начала плавания'] - origin_time).total_seconds() / 60),
                    'coords': list(start_place_grid_coord),
                    'edge': -1, # из таблицы граф данных
                    'status': 0, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
                    'speed': 0,
                    'assigned_icebracker':None,
                 }
            ]}
        initial_ships_list.append(ship)
    
    initial_icebrakers_list = [{
        "name_icebracker": "50 лет Победы",
        "date_start": "2022-03-01", # 00:00:00
        "class_ship": 'Arc 9',
        "speed_ship": 22,
        "route": [
            { 
                'time': 0, # 1 минута
                'grid_coords': list(points_grid_coords_dict[27]), # координаты секти (row, columns) ?
                'status': 0, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': 0, # обратное преобразование Германа 
                'assigned_ships': None, 
             },
        ]
    },
    {
        "name_icebracker": "Ямал",
        "date_start": "2022-03-01", # 00:00:00
        "class_ship": 'Arc 9',
        "speed_ship": 21,
        "route": [
            { 
                'time': 0, # 1 минута
                'grid_coords': list(points_grid_coords_dict[41]), # координаты секти (row, columns) ?
                'status': 0, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': 0, # обратное преобразование Германа 
                'assigned_ships': None, 
             },
        ]
    },{
        "name_icebracker": "Таймыр",
        "date_start": "2022-03-01", # 00:00:00
        "class_ship": 'Arc 9',
        "speed_ship": 18.5,
        "route": [
            { 
                'time': 0, # 1 минута
                'grid_coords': list(points_grid_coords_dict[16]), # координаты секти (row, columns) ?
                'status': 0, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': 0, # обратное преобразование Германа 
                'assigned_ships': None, 
             },
        ]
    },{
        "name_icebracker": "Вайгач",
        "date_start": "2022-03-01", # 00:00:00
        "class_ship": 'Arc 9',
        "speed_ship": 18.5,
        "route": [
            { 
                'time': 0, # 1 минута
                'grid_coords': list(points_grid_coords_dict[6]), # координаты секти (row, columns) ?
                'status': 0, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': 0, # обратное преобразование Германа 
                'assigned_ships': None, 
             },
        ]
    },]
    
    return initial_ships_list, initial_icebrakers_list


In [208]:
initial_ships_list, initial_icebraekers_list = make_initial_result_dicts(points_grid_coords_dict, path='data/Расписание движения судов.xlsx')

In [252]:
initial_icebraekers_list[:4]

NameError: name 'initial_icebraekers_list' is not defined

In [18]:
def make_icebreaker_availability_list(initial_icebrakers_list):
    
    icebreaker_availability_list = []
    for icebraker in initial_icebrakers_list:
        record = {
            'name_icebracker': icebraker['name_icebracker'],
            'date_start': icebraker['date_start'],
            'class_ship': icebraker['class_ship'],
            'speed_ship': icebraker['speed_ship'],
            'time_place_availible': [icebraker['route'][0]['time'], icebraker['route'][0]['grid_coords']],
            'assigned_ships': icebraker['route'][0]['assigned_ships']
        }
        icebreaker_availability_list.append(record)
    return icebreaker_availability_list

In [214]:
icebraker_availability_list = make_icebraker_availability_list(initial_icebrakers_list)

In [215]:
icebraker_availability_list

[{'name_icebracker': '50 лет Победы',
  'date_start': '2022-03-01',
  'class_ship': 'Arc 9',
  'speed_ship': 22,
  'time_place_availible': [0, [200, 31]],
  'assigned_ships': None},
 {'name_icebracker': 'Ямал',
  'date_start': '2022-03-01',
  'class_ship': 'Arc 9',
  'speed_ship': 21,
  'time_place_availible': [0, [26, 23]],
  'assigned_ships': None},
 {'name_icebracker': 'Таймыр',
  'date_start': '2022-03-01',
  'class_ship': 'Arc 9',
  'speed_ship': 18.5,
  'time_place_availible': [0, [78, 42]],
  'assigned_ships': None},
 {'name_icebracker': 'Вайгач',
  'date_start': '2022-03-01',
  'class_ship': 'Arc 9',
  'speed_ship': 18.5,
  'time_place_availible': [0, [67, 48]],
  'assigned_ships': None}]

In [222]:
edges_list[:5]

[(44, 15, (27, 48), (46, 54)),
 (10, 11, (71, 60), (59, 78)),
 (18, 39, (191, 42), (195, 44)),
 (13, 16, (77, 60), (78, 42)),
 (10, 13, (71, 60), (77, 60))]

In [19]:
from typing import Tuple
import numpy as np

from transform_coord_to_grid import load_table_lat_lon, compute_grid_coord
from speed_map import read_velocity, compute_speed_by_date_for_ship


def add_queen_edges(G, matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    for i in range(rows):
        for j in range(cols):
            if matrix[i][j] > 0:
                # Adding horizontal and vertical edges (these are already present in grid_2d_graph)
                # Adding diagonal edges
                for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < rows and 0 <= nj < cols and matrix[ni][nj] > 0:
                        G.add_edge((i, j), (ni, nj))


def get_route(
        speed_grid: pd.DataFrame,
        start_coords: Tuple[int, int], end_coords: Tuple[int, int]
):
    def get_time(sheet, p):
        if sheet.iloc[p[0], p[1]] > 0:
            return sheet.iloc[p[0], p[1]]
        else:
            return 0
    
    start_coords = tuple(start_coords)
    end_coords = tuple(end_coords)
    matrix = speed_grid.values
    G = nx.grid_2d_graph(len(matrix), len(matrix[0]))
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] < 0:
                G.remove_node((i, j))
    add_queen_edges(G, matrix)
    try:
        path = nx.astar_path(G, start_coords, end_coords)
    except nx.NetworkXNoPath:
        return None, 0
    except nx.NodeNotFound:
        return None, 0

    time_required = sum([get_time(speed_grid, p) for p in path])
    path = [[p[0], p[1], get_time(speed_grid, p)] for p in path]
    return path, time_required


def search_coord(speed_grid, grid_coords):

    if speed_grid.iloc[grid_coords[0], grid_coords[1]] > 0:
        return grid_coords
    
    indices = np.where(speed_grid > 0)
    distances = np.sqrt((indices[0] - grid_coords[0])**2 + (indices[1] - grid_coords[1])**2)
    nearest_index = np.argmin(distances)
    x, y = indices[0][nearest_index], indices[1][nearest_index]
    return (x, y)


def get_all_edges_coords(xls_graph, xls_ice):
    
    points = pd.read_excel(xls_graph, sheet_name='points', header=0)
    edges = pd.read_excel(xls_graph, sheet_name='edges', header=0)
    
    table_lat, table_lon = load_table_lat_lon(path_to_excel='data/IntegrVelocity.xlsx')

    speed_grid = compute_speed_by_date_for_ship(xls_ice, date="15-03-2020", name_ship='Ямал', class_ship='Arc 9', speed_ship=21.0, icebreaker=True)
    
    edges_list = []
    for _, row in edges.iterrows():
        
        start_point_id = row['start_point_id']
        end_point_id = row['end_point_id']
        
        start_point_lat = points['latitude'][points['point_id'] == start_point_id].values[0]
        start_point_lon = points['longitude'][points['point_id'] == start_point_id].values[0]
        
        end_point_lat = points['latitude'][points['point_id'] == end_point_id].values[0]
        end_point_lon = points['longitude'][points['point_id'] == end_point_id].values[0]
        
        start_grid_coord = compute_grid_coord(table_lat, table_lon, start_point_lat, start_point_lon, 0.2)
        start_grid_coord = search_coord(speed_grid, start_grid_coord)
        
        end_grid_coord = compute_grid_coord(table_lat, table_lon, end_point_lat, end_point_lon, 0.2)
        end_grid_coord = search_coord(speed_grid, end_grid_coord)
        
        edges_list.append((start_point_id, end_point_id, start_grid_coord, end_grid_coord))
    return edges_list


def get_points_grid_coords(xls_graph, xls_ice):
    
    points = pd.read_excel(xls_graph, sheet_name='points', header=0)
    table_lat, table_lon = load_table_lat_lon(path_to_excel='data/IntegrVelocity.xlsx')

    speed_grid = compute_speed_by_date_for_ship(xls_ice, date="15-03-2020", name_ship='Ямал', class_ship='Arc 9', speed_ship=21.0, icebreaker=True)

    points_grid_coords_dict = {}
    for _, row in points.iterrows():
        
        point_id = row['point_id']
        point_lat = row['latitude']
        point_lon = row['longitude']
    
        point_grid_coord = compute_grid_coord(table_lat, table_lon, point_lat, point_lon, 0.2)
        point_grid_coord = search_coord(speed_grid, point_grid_coord)
        
        points_grid_coords_dict[point_id] = point_grid_coord
    return points_grid_coords_dict


def make_edges_for_dijkstra(edges_list, speed_grid):
    
    edges_list_dijkstra = []
    for edge in edges_list:
        route, time_required = get_route(speed_grid, edge[2], edge[3])
        if not route:
            weight = 100000
        else:
            weight = time_required
        edges_list_dijkstra.append([edge[0], edge[1], weight])
    return edges_list_dijkstra
    
    
def get_initial_route(edges_list, speed_grid, start_point_id, end_point_id):
    
    edges_list_dijkstra = make_edges_for_dijkstra(edges_list, speed_grid)
    
    edges_dict_dijkstra = {f'{edge[0]}_{edge[1]}': edge[2] for edge in edges_list_dijkstra}
    for edge in edges_list_dijkstra:
        if f'{edge[1]}_{edge[0]}' not in edges_dict_dijkstra:
            edges_dict_dijkstra[f'{edge[1]}_{edge[0]}'] = edge[2]
            
    graph = Graph(edges=edges_list_dijkstra, bi_directional=True)
    finder = DijkstraFinder()
    path, runs = finder.find_path(graph.node(start_point_id), graph.node(end_point_id), graph)

    edges_to_visit = ([[start_node.node_id, end_node.node_id, edges_dict_dijkstra[f'{start_node.node_id}_{end_node.node_id}']] 
                       for start_node, end_node in zip(path[:-1], path[1:])])

    return edges_to_visit


def get_edge_id(start_point, end_point, xls_graph):
    
    edges = pd.read_excel(xls_graph, sheet_name='edges', header=0)
    try:
        result = edges['id'][(edges['start_point_id'] == start_point) & (edges['end_point_id'] == end_point)].values[0]
    except:
        result = edges['id'][(edges['start_point_id'] == end_point) & (edges['end_point_id'] == start_point)].values[0]
    return result


def assign_and_update_icebraker(xls_ice, ship_result_dict, icebreaker_availability_list, 
                     start_grid_coord, end_grid_coord, icebreakers_result_list, speed_map_with_icebreaker):
    const = 1.852
    date = ship_result_dict["date_start"]
    name_ship = ship_result_dict["name_ship"]
    class_ship = ship_result_dict["class_ship"]
    speed_ship = ship_result_dict["speed_ship"]   

    # speed_map_with_icebreaker = compute_speed_by_date_for_ship(excel, date, name_ship, class_ship, speed_ship, icebreaker=True)
    
    time_to_arrive = np.inf
    route_to_arrive = None
    assigned_icebreaker = None
    
    for i, icebreaker in enumerate(icebreaker_availability_list):
        
        #compute time required to icebreaker to reach the ship
        speed_grid_icebraker = compute_speed_by_date_for_ship(xls_ice, date, icebreaker['name_icebracker'], 
                                                            icebreaker['class_ship'], icebreaker['speed_ship'], icebreaker=False)
        
        icebreaker_route, icebreaker_time_required = get_route(speed_grid_icebraker, icebreaker['time_place_availible'][1], list(start_grid_coord))
        
        icebreaker_time_required = int(icebreaker_time_required * 60)
        
        time_to_finish_current_task = icebreaker['time_place_availible'][0] - ship_result_dict['route'][-1]['time']
        time_to_finish_current_task = 0 if time_to_finish_current_task < 0 else time_to_finish_current_task
        
        icebreaker_time_required += time_to_finish_current_task
        
        if icebreaker_route and time_to_arrive > icebreaker_time_required:
            time_to_arrive = icebreaker_time_required
            assigned_icebreaker = icebreaker
            route_to_arrive = icebreaker_route
            # print(icebreaker['name_icebracker'])
            # print(icebreaker['time_place_availible'])
            i_to_update = i
    
    #update icebraker_availability_list
    route_with_icebreaker, time_to_travel_with_icebreaker = get_route(speed_map_with_icebreaker, start_grid_coord, end_grid_coord)
    assigned_icebreaker['time_place_availible'] = [ship_result_dict['route'][-1]['time'] + time_to_arrive + time_to_travel_with_icebreaker, end_grid_coord]
    assigned_icebreaker['assigned_ships'] = [name_ship]
    icebreaker_availability_list[i_to_update] = assigned_icebreaker
    
    #update icebreakers_result_list
    icebreaker_to_update = icebreakers_result_list[i_to_update]
    
    #add path to reach assigned ship
    # print(route_to_arrive)
    for grid_cell in route_to_arrive:
        minutes = int((grid_cell[2] * 60))
        coords = [grid_cell[0], grid_cell[1]]
        status = 1
        speed = grid_cell[2] * 25 / const
        assigned_ships = None
        time = icebreaker_to_update["route"][-1]["time"]

        for minute in range(time + 1, time + minutes + 1):
            record = { 
                'time': minute,
                'grid_coords': coords,
                'status': status, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': speed,
                'assigned_ships': assigned_ships,
            }
            icebreaker_to_update["route"].append(record)
            
    #add path to escort assigned ship
    for grid_cell in route_with_icebreaker:
        minutes = int((grid_cell[2] * 60))
        coords = [grid_cell[0], grid_cell[1]]
        status = 2
        speed = grid_cell[2] * 25 / const
        assigned_ships = [name_ship]
        time = icebreaker_to_update["route"][-1]["time"]

        for minute in range(time + 1, time + minutes + 1):
            record = { 
                'time': minute,
                'grid_coords': coords,
                'status': status, # 0 - стоит; 1 - едет пустой; 2 - едет с судоном
                'speed': speed,
                'assigned_ships': assigned_ships,
            }
            icebreaker_to_update["route"].append(record)
            
    icebreakers_result_list[i_to_update] = icebreaker_to_update
    
    return (icebreaker_availability_list, 
            assigned_icebreaker['name_icebracker'], 
            time_to_arrive, time_to_travel_with_icebreaker, 
            route_with_icebreaker, icebreakers_result_list)
    
    
    
def add_records(xls_graph, ship_result_dict, start_point, end_point, 
                 ice_path, time_required, icebreaker_availability,
                 icebreaker=None, idle_time=None, icebreaker_result_dict=None):
    
    const = 1.852
    
    if not icebreaker:
        for grid_cell in ice_path:
            
            minutes = int((grid_cell[2] * 60))
            coords = [grid_cell[0], grid_cell[1]]
            edge = get_edge_id(start_point, end_point, xls_graph)
            status = 1
            speed = grid_cell[2] * 25 / const
            assigned_icebreaker = None
            time = ship_result_dict["route"][-1]["time"]

            for minute in range(time + 1, time + minutes + 1):
                record = { 
                    'time': minute,
                    'coords': coords,
                    'edge': edge, # из таблицы граф данных
                    'status': status, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
                    'speed': speed,
                    'assigned_icebracker': assigned_icebreaker,
                }
                ship_result_dict["route"].append(record)
    
    if idle_time:
        minutes = int(idle_time) #int((idle_time * 60))
        coords = ship_result_dict["route"][-1]["coords"]
        edge = ship_result_dict["route"][-1]["edge"]
        status = 0
        speed = 0
        assigned_icebreaker = None
        time = ship_result_dict["route"][-1]["time"]
        
        for minute in range(time + 1, time + minutes + 1):
            record = { 
                'time': minute,
                'coords': coords,
                'edge': edge, # из таблицы граф данных
                'status': status, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
                'speed': speed,
                'assigned_icebracker': assigned_icebreaker,
            }
            ship_result_dict["route"].append(record)
            
    if icebreaker:
        for grid_cell in ice_path:
            
            minutes = int((grid_cell[2] * 60))
            coords = [grid_cell[0], grid_cell[1]]
            edge = get_edge_id(start_point, end_point, xls_graph)
            status = 2
            speed = grid_cell[2] * 25 / const
            assigned_icebreaker = icebreaker
            time = ship_result_dict["route"][-1]["time"]

            for minute in range(time + 1, time + minutes + 1):
                record = { 
                    'time': minute,
                    'coords': coords,
                    'edge': edge, # из таблицы граф данных
                    'status': status, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
                    'speed': speed,
                    'assigned_icebracker': assigned_icebreaker,
                }
                ship_result_dict["route"].append(record)
    return ship_result_dict
        


def clarify_route(xls_graph, xls_ice, edges_to_visit, edges_list, 
                  speed_grid_solo, points_grid_coords_dict,
                  icebreaker_availability_list,
                  ship_result_dict, icebreakers_result_list, speed_grid_with_icebraker=None):
    
    ship_route = []
    for edge in edges_to_visit:
        start_grid_coord = points_grid_coords_dict[edge[0]]
        end_grid_coord = points_grid_coords_dict[edge[1]]
        
        route, time_required = get_route(speed_grid_solo, start_grid_coord, end_grid_coord)
        if time_required and time_required < 100000:
            ship_result_dict = add_records(xls_graph, ship_result_dict, edge[0], edge[1], route, time_required, None, None)
            ship_route.extend(route)
        else:
            res = assign_and_update_icebraker(xls_ice, ship_result_dict, icebreaker_availability_list, 
                                              start_grid_coord, end_grid_coord, icebreakers_result_list, 
                                              speed_grid_with_icebraker)
            
            (icebreaker_availability_list, assigned_icebreaker, 
             time_to_arrive, time_to_travel_with_icebraker, 
             route_with_icabreaker, icebreakers_result_list) = res
                                                                                                                              
            ship_result_dict = add_records(xls_graph, ship_result_dict, 
                                           edge[0], edge[1], 
                                           route_with_icabreaker, time_to_travel_with_icebraker,
                                          time_to_arrive, assigned_icebreaker, idle_time=time_to_arrive)
            ship_route.extend(route_with_icabreaker)
            
    ship_result_dict['ship_route'] = ship_route
    return ship_result_dict, icebreakers_result_list, icebreaker_availability_list

In [20]:
from tqdm import tqdm

def get_id_from_name(xls_graph, place_name):
    
    points = pd.read_excel(xls_graph, sheet_name='points', header=0)
    points['point_name'] = points['point_name'].apply(lambda x: str(x).lower())
    
    place_name = place_name.lower()
    if place_name == 'восточно-сибирское 1':
        place_name = 'Восточно-Сибирское - 1 (восток)'.lower()
    elif place_name == 'восточно-сибирское 3':
        place_name = 'Восточно-Сибирское - 3 (север)'.lower()

    place_id = points['point_id'][points['point_name'] == place_name].iloc[0]
    return place_id

In [21]:
def make_schedule():
    
    path_schedule = 'data/Расписание движения судов.xlsx'
    xls_schedule = pd.ExcelFile(path_schedule)
    
    path_graph = 'data/ГрафДанные.xlsx'
    xls_graph = pd.ExcelFile(path_graph)
    
    path_ice = 'data/IntegrVelocity.xlsx'
    xls_ice = pd.ExcelFile(path_ice)    
    
    points_grid_coords_dict = get_points_grid_coords(xls_graph, xls_ice)
    edges_list = get_all_edges_coords(xls_graph, xls_ice)
    initial_ships_list, initial_icebreakers_list = make_initial_result_dicts(points_grid_coords_dict, xls_schedule, xls_graph)
    icebreaker_availability_list = make_icebreaker_availability_list(initial_icebreakers_list)
    
    applications = pd.read_excel(xls_schedule, sheet_name='Лист1', header=0)
    applications = applications.iloc[:42]
    
    for i, application in tqdm(list(applications.iterrows())):
        
        ship_result_dict = initial_ships_list[i]
        date = str(application['Дата начала плавания'].date())
        start_point_id = get_id_from_name(xls_graph, application['Пункт начала плавания'])
        end_point_id = get_id_from_name(xls_graph, application['Пункт окончания плавания'])      
        name_ship = application['Название судна']     
        class_ship = application['Ледовый класс']     
        speed_ship = application['''Скорость, узлы
(по чистой воде)''']  

        speed_grid_with_icebreaker = compute_speed_by_date_for_ship(xls_ice, date, name_ship, class_ship, speed_ship, icebreaker=True)
        speed_grid_solo = compute_speed_by_date_for_ship(xls_ice, date, name_ship, class_ship, speed_ship, icebreaker=False)
        
        try:
            edges_to_visit = get_initial_route(edges_list, speed_grid_with_icebreaker, start_point_id, end_point_id)

            ship_result_dict, icebreakers_result_list, icebreaker_availability_list = clarify_route(xls_graph, xls_ice, edges_to_visit, edges_list, 
                                                                                      speed_grid_solo, points_grid_coords_dict,
                                                                                      icebreaker_availability_list,
                                                                                      ship_result_dict, initial_icebreakers_list, speed_grid_with_icebreaker)
            initial_ships_list[i] = ship_result_dict
        except Exception as e:
            print(e)
            continue
        
        
    return initial_ships_list, initial_icebreakers_list

In [34]:
scheduled_ships_list, scheduled_icebreakers_list = make_schedule()

100%|██████████| 2/2 [02:51<00:00, 85.60s/it]


### END

In [33]:
6644/60/24

4.613888888888889

In [32]:
len(scheduled_ships_list[0]['route'])

6644

In [35]:
len(scheduled_ships_list[0]['route'])

6644

In [31]:
import pickle

with open('data/example_icebreaker_full.p', 'wb') as fp:
    pickle.dump(scheduled_icebreakers_list[3], fp)

In [14]:
scheduled_ships_list

[{'name_ship': 'ДЮК II',
  'date_start': '2022-03-01',
  'class_ship': 'Arc 5',
  'speed_ship': 15,
  'route': [{'time': 0,
    'coords': [59, 78],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'САРМАТ',
  'date_start': '2022-03-02',
  'class_ship': 'Arc 4',
  'speed_ship': 15,
  'route': [{'time': 1440,
    'coords': [64, 66],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'EDUARD TOLL',
  'date_start': '2022-03-04',
  'class_ship': 'Arc 7',
  'speed_ship': 15,
  'route': [{'time': 4320,
    'coords': [64, 66],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'GEORGIY USHAKOV',
  'date_start': '2022-03-07',
  'class_ship': 'Arc 7',
  'speed_ship': 15,
  'route': [{'time': 8640,
    'coords': [46, 31],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'RUDOLF SAMOYLOVICH',
  'date_

In [424]:
print(scheduled_ships_list[3]['ship_route'])
scheduled_ships_list[3]['route'][78000:78500]

[[46, 31, 1.111], [47, 31, 1.111], [48, 31, 1.111], [49, 31, 1.111], [50, 31, 1.111], [51, 31, 1.111], [52, 31, 1.111], [53, 31, 1.111], [54, 31, 1.111], [55, 31, 1.111], [56, 31, 1.111], [57, 31, 1.111], [58, 31, 1.111], [59, 31, 1.111], [60, 31, 1.111], [61, 31, 1.111], [62, 32, 1.111], [63, 33, 1.111], [64, 34, 1.111], [65, 35, 1.111], [66, 36, 1.111], [67, 37, 1.111], [68, 38, 1.111], [68, 38, 1.111], [69, 38, 1.111], [70, 38, 1.111], [71, 38, 1.111], [72, 38, 1.111], [73, 38, 1.111], [74, 38, 1.111], [75, 39, 1.111], [76, 40, 1.111], [77, 41, 1.111], [78, 42, 1.111], [78, 42, 1.111], [79, 42, 1.111], [80, 42, 1.111], [81, 42, 1.111], [82, 42, 1.111], [83, 42, 1.111], [84, 42, 1.111], [85, 42, 1.111], [86, 42, 1.111], [87, 42, 1.111], [88, 42, 1.111], [89, 42, 0.667], [90, 42, 0.667], [91, 42, 1.111], [92, 42, 1.111], [93, 42, 0.167], [94, 42, 0.167], [95, 42, 0.167], [96, 43, 0.167], [97, 44, 0.167], [98, 45, 0.167], [99, 46, 0.167], [100, 47, 0.167], [101, 48, 0.167], [102, 49, 0

[{'time': 86640,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86641,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86642,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86643,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86644,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86645,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86646,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86647,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 0,
  'assigned_icebracker': None},
 {'time': 86648,
  'coords': [159, 45],
  'edge': 13,
  'status': 0,
  'speed': 

In [401]:
initial_ships_list[0]['ship_route']

[[59, 78, 0.778],
 [59, 77, 0.778],
 [59, 76, 0.778],
 [59, 75, 0.778],
 [59, 74, 0.778],
 [60, 73, 0.889],
 [60, 72, 0.889],
 [61, 71, 0.778],
 [62, 70, 0.778],
 [63, 69, 0.778],
 [64, 68, 0.778],
 [64, 67, 0.778],
 [65, 66, 0.778],
 [66, 65, 0.778],
 [67, 64, 0.778],
 [68, 63, 0.778],
 [69, 62, 1.111],
 [70, 61, 0.778],
 [71, 60, 1.111],
 [71, 60, 1.111],
 [71, 59, 1.111],
 [71, 58, 1.111],
 [70, 57, 1.111],
 [69, 57, 1.111],
 [68, 57, 1.111],
 [67, 57, 1.111],
 [66, 57, 1.111],
 [65, 57, 1.111],
 [64, 57, 1.111],
 [63, 57, 1.111],
 [62, 57, 1.111],
 [61, 57, 1.111],
 [60, 57, 1.111],
 [59, 57, 1.111],
 [58, 57, 1.111],
 [57, 57, 1.111],
 [56, 57, 1.111],
 [55, 57, 1.111],
 [54, 57, 1.111],
 [53, 57, 1.111],
 [52, 57, 1.111],
 [51, 57, 1.111],
 [50, 57, 1.111],
 [49, 57, 1.111],
 [48, 56, 1.111],
 [47, 55, 1.111],
 [46, 54, 1.111],
 [46, 54, 1.111],
 [46, 53, 1.111],
 [46, 52, 1.111],
 [46, 51, 1.111],
 [45, 50, 1.111],
 [44, 49, 1.111],
 [44, 48, 1.111],
 [43, 47, 1.111],
 [42, 46, 

In [195]:
        
initial_ships_list =  make_initial_result_dicts(points_grid_coords_dict, path='data/Расписание движения судов.xlsx')
    

In [196]:
initial_ships_list

[{'name_ship': 'ДЮК II',
  'date_start': '2022-03-01 00:00:00',
  'route': [{'time': 0,
    'coords': [59, 78],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'САРМАТ',
  'date_start': '2022-03-02 00:00:00',
  'route': [{'time': 1440,
    'coords': [64, 66],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'EDUARD TOLL',
  'date_start': '2022-03-04 00:00:00',
  'route': [{'time': 4320,
    'coords': [64, 66],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'GEORGIY USHAKOV',
  'date_start': '2022-03-07 00:00:00',
  'route': [{'time': 8640,
    'coords': [46, 31],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigned_icebracker': None}]},
 {'name_ship': 'RUDOLF SAMOYLOVICH',
  'date_start': '2022-03-08 00:00:00',
  'route': [{'time': 10080,
    'coords': [59, 78],
    'edge': -1,
    'status': 0,
    'speed': 0,
    'assigne

In [257]:
path='data/Расписание движения судов.xlsx'
    
xls = pd.ExcelFile(path)
applications = pd.read_excel(xls, sheet_name='Лист1', header=0)
initial_day = applications['Дата начала плавания'].iloc[0]

print(int((applications['Дата начала плавания'].iloc[1] - initial_day).total_seconds() / 60))
print(initial_day)
print(type(applications['''Скорость, узлы
(по чистой воде)'''].iloc[0]))

1440
2022-03-01 00:00:00
<class 'int'>


In [274]:
ship_result_dict = {
    "name_ship": "ДЮК II",
    "date_start": "2024-01-01",
    "route": [
        { 
            'time': 0,
            'coords': [68, 38],
            'edge': 65, # из таблицы граф данных
            'status': 0, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
            'speed': float,
            'assigned_icebracker':None,
         }
    ]
}

In [280]:
# ship_result_dict = clarify_route(edges_to_visit, edges_list,
#                                 sheet, points_grid_coords_dict, 
#                                 ship_result_dict)
path_graph = 'data/ГрафДанные.xlsx'
xls_graph = pd.ExcelFile(path_graph)
    
ship_result_dict = clarify_route(xls_graph, edges_to_visit, edges_list, 
                                  sheet, points_grid_coords_dict,
                                  icebreaker_availability_list,
                                  ship_result_dict, speed_grid_with_icebraker=None)

In [281]:
ship_result_dict

{'name_ship': 'ДЮК II',
 'date_start': '2024-01-01',
 'route': [{'time': 0,
   'coords': [68, 38],
   'edge': 65,
   'status': 0,
   'speed': float,
   'assigned_icebracker': None},
  {'time': 1,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 2,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 3,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 4,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 5,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 6,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 7,
   'co

In [262]:
type(ship_result_dict)

dict

In [282]:
import pickle

with open('data/example_ship.p', 'wb') as fp:
    pickle.dump(ship_result_dict, fp)

In [266]:
import pickle

with open('data/example_ship.p', 'rb') as fp:
    data = pickle.load(fp)

In [267]:
data

{'name_ship': 'ДЮК II',
 'date_start': '2024-01-01',
 'route': [{'time': 0,
   'coords': [68, 38],
   'edge': 65,
   'status': 0,
   'speed': float,
   'assigned_icebracker': None},
  {'time': 1,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 2,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 3,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 4,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 5,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 6,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 7,
   'co

In [258]:
ship_result_dict

{'name_ship': 'ДЮК II',
 'date_start': '2024-01-01',
 'route': [{'time': 0,
   'coords': [68, 38],
   'edge': 65,
   'status': 0,
   'speed': float,
   'assigned_icebracker': None},
  {'time': 1,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 2,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 3,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 4,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 5,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 6,
   'coords': [68, 38],
   'edge': 65,
   'status': 1,
   'speed': 19.006479481641467,
   'assigned_icebracker': None},
  {'time': 7,
   'co

In [99]:
get_edge_id(start_point=16, end_point=21, path='data/ГрафДанные.xlsx')

65

In [225]:
points_grid_coords_dict = get_points_grid_coords(path='data/ГрафДанные.xlsx')

get_route(sheet, points_grid_coords_dict[21], points_grid_coords_dict[16])

([[68, 38, 1.408],
  [69, 38, 1.408],
  [70, 38, 1.408],
  [71, 38, 1.408],
  [72, 39, 1.408],
  [73, 40, 1.408],
  [74, 41, 1.408],
  [75, 41, 1.408],
  [76, 41, 1.408],
  [77, 41, 1.408],
  [78, 42, 1.408]],
 15.487999999999996)

In [226]:
edges_to_visit

[[21, 16, 15.487999999999996],
 [16, 20, 100000],
 [20, 19, 100000],
 [19, 8, 100000],
 [8, 9, 100000],
 [9, 24, 23.373000000000005],
 [24, 22, 100000],
 [22, 23, 100000]]

In [22]:
edges_list = get_all_edges_coords()

In [155]:
edges_list[:5]

[(44, 15, (27, 48), (46, 54)),
 (10, 11, (71, 60), (59, 78)),
 (18, 39, (191, 42), (195, 44)),
 (13, 16, (77, 60), (78, 42)),
 (10, 13, (71, 60), (77, 60))]

In [157]:
points_grid_coords_dict

{0: (78, 66),
 1: (79, 84),
 2: (34, 38),
 3: (42, 58),
 4: (46, 31),
 5: (33, 3),
 6: (67, 48),
 7: (92, 55),
 8: (115, 55),
 9: (125, 67),
 10: (71, 60),
 11: (59, 78),
 12: (123, 58),
 13: (77, 60),
 14: (77, 68),
 15: (46, 54),
 16: (78, 42),
 17: (195, 20),
 18: (191, 42),
 19: (111, 54),
 20: (107, 54),
 21: (68, 38),
 22: (150, 60),
 23: (159, 56),
 24: (139, 67),
 25: (64, 66),
 26: (196, 47),
 27: (200, 31),
 28: (180, 36),
 29: (9, 39),
 30: (159, 45),
 31: (36, 56),
 32: (115, 73),
 33: (178, 51),
 34: (61, 57),
 35: (66, 69),
 36: (104, 59),
 37: (230, 23),
 38: (237, 54),
 39: (195, 44),
 40: (141, 59),
 41: (26, 23),
 42: (147, 50),
 43: (86, 61),
 44: (27, 48),
 45: (223, 16),
 46: (243, 90)}

In [ ]:
{
    "name_ship": "ДЮК II",
    "date_start": "2024-01-01"
    "route": [
        { 
            'time': 0,
            'coords': [x, y],
            'edge': int, # из таблицы граф данных
            'status': int, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
            'speed': float,
            'assigned_icebracker': Option[None, int],
         },
    ]
}

In [236]:
edges_list[:4]

[(44, 15, (27, 48), (46, 54)),
 (10, 11, (71, 60), (59, 78)),
 (18, 39, (191, 42), (195, 44)),
 (13, 16, (77, 60), (78, 42))]

In [227]:
edges_to_visit = get_initial_route(edges_list, sheet, 21, 23)

In [228]:
edges_to_visit

[[21, 16, 15.487999999999996],
 [16, 20, 100000],
 [20, 19, 100000],
 [19, 8, 100000],
 [8, 9, 100000],
 [9, 24, 23.373000000000005],
 [24, 22, 100000],
 [22, 23, 100000]]

In [23]:
edges_list

[(44, 15, (27, 48), (46, 54)),
 (10, 11, (71, 60), (59, 78)),
 (18, 39, (191, 42), (195, 44)),
 (13, 16, (77, 60), (78, 42)),
 (10, 13, (71, 60), (77, 60)),
 (21, 5, (68, 38), (33, 3)),
 (4, 5, (46, 31), (33, 3)),
 (45, 37, (223, 16), (230, 23)),
 (13, 7, (77, 60), (92, 55)),
 (9, 24, (125, 67), (139, 67)),
 (27, 18, (200, 31), (191, 42)),
 (28, 33, (180, 36), (178, 51)),
 (40, 22, (141, 59), (150, 60)),
 (42, 30, (147, 50), (159, 45)),
 (31, 3, (36, 56), (42, 58)),
 (0, 43, (78, 66), (86, 61)),
 (12, 24, (123, 58), (139, 67)),
 (12, 42, (123, 58), (147, 50)),
 (15, 2, (46, 54), (34, 38)),
 (10, 35, (71, 60), (66, 69)),
 (9, 32, (125, 67), (115, 73)),
 (2, 3, (34, 38), (42, 58)),
 (0, 1, (78, 66), (79, 84)),
 (33, 18, (178, 51), (191, 42)),
 (40, 42, (141, 59), (147, 50)),
 (4, 41, (46, 31), (26, 23)),
 (8, 12, (115, 55), (123, 58)),
 (2, 29, (34, 38), (9, 39)),
 (15, 3, (46, 54), (42, 58)),
 (12, 40, (123, 58), (141, 59)),
 (19, 8, (111, 54), (115, 55)),
 (28, 30, (180, 36), (159, 45)

In [242]:
def get_route(speed_grid: pd.DataFrame, 
              start_coords: Tuple[int, int], 
              end_coords: Tuple[int, int]):
    
    def get_time(sheet, p):
        sheet = sheet.T
        if sheet.iloc[p[0], p[1]] > 0:
            return sheet.iloc[p[0], p[1]]
        else:
            return 0
        
    matrix = speed_grid.T.values
    grid = Grid(matrix=matrix)

    start = grid.node(*start_coords)
    end = grid.node(*end_coords)
    
    finder = AStarFinder(diagonal_movement=DiagonalMovement.always)
    path, runs = finder.find_path(start, end, grid)
    
    path = [[step.x, step.y] for step in path]
    time_required = sum([get_time(sheet, p) for p in path])
    path = [[p[0], p[1], get_time(sheet, p)] for p in path]
    return path, time_required

for edge in edges_list:

    path, time_required = get_route(sheet, edge[2], edge[3])
    print(time_required)

26.752000000000006
0
0
0
0
0
0
0
0
0
0
0
0
0
14.079999999999997
0
0
0
19.712
0
0
29.56800000000001
0
0
0
26.752000000000006
0
0
4.224
0
0
0
0
32.94800000000001
0
0
14.361999999999997
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15.487999999999996
0
0
0
0
0
0.422
0
0
19.712
0
0


In [250]:
# noT

def get_route(speed_grid: pd.DataFrame, 
              start_coords: Tuple[int, int], 
              end_coords: Tuple[int, int]):
    
    def get_time(sheet, p):
        if sheet.iloc[p[0], p[1]] > 0:
            return sheet.iloc[p[0], p[1]]
        else:
            return 0
        
    matrix = speed_grid.values
    
    grid = Grid(matrix=matrix)
    
    try:
        start = grid.node(*start_coords)
    except Exception as e:
        print(e)
        print(start_coords)
        print(grid.height, grid.width)
    
    try:
        end = grid.node(*end_coords)
    except:
        print(end_coords)
        print(grid.height, grid.width)
        
    finder = AStarFinder(diagonal_movement=DiagonalMovement.always)
    path, runs = finder.find_path(start, end, grid)
    
    path = [[step.x, step.y] for step in path]
    time_required = sum([get_time(sheet, p) for p in path])
    path = [[p[0], p[1], get_time(sheet, p)] for p in path]
    return path, time_required

for edge in edges_list:

    path, time_required = get_route(sheet, edge[2], edge[3])
    print(time_required)

21.965000000000003
0
0
0
0
0
0
list index out of range
(223, 16)
269 217
(230, 23)
269 217


UnboundLocalError: local variable 'start' referenced before assignment

In [234]:
path, time_required

([[27, 48, 1.408],
  [28, 48, 1.408],
  [29, 48, 1.408],
  [30, 49, 1.408],
  [31, 50, 1.408],
  [32, 50, 1.408],
  [33, 50, 1.408],
  [34, 50, 1.408],
  [35, 50, 1.408],
  [36, 50, 1.408],
  [37, 50, 1.408],
  [38, 50, 1.408],
  [39, 50, 1.408],
  [40, 50, 1.408],
  [41, 50, 1.408],
  [42, 51, 1.408],
  [43, 51, 1.408],
  [44, 52, 1.408],
  [45, 53, 1.408],
  [46, 54, 1.408]],
 28.160000000000007)

In [18]:
path = [[10, 4],[10, 4],[10, 4],[100, 4], [58, 77]]

t = sum([sheet.iloc[p[0], p[1]] for p in path])

In [20]:
sheet.iloc[58, 77]

0.211

In [ ]:
{
    "name_ship": "ДЮК II",
    "date_start": "2024-01-01"
    "route": [
        { 
            'time': 0,
            'coords': [x, y],
            'edge': int, # из таблицы граф данных
            'status': int, # 0 - стоит; 1 - плывет самостоятельно; 2 - плывет с ледоколом
            'speed': float,
            'assigned_icebracker': Option[None, int],
         },
    ]
}

In [ ]:
ship    time 0                                15                    30

DYK2         {
              'coords': [x, y]
              'edge': int,
              'status': int,
              'speed': float,
              'icebracker': Option[None, int],
             }

In [ ]:
{ 'coords': [x, y],
              'edge': int,
              'status': int,
              'speed': float,
              'icebracker': Option[None, int],
             }